# K2-Photometry

Here we

1. Generate detrended K2 photometry
2. Remove outliers
3. Whiten light-curve
4. Refit and resample light curve with linear emphem

In [1]:
from io import StringIO as sio
import everest
from everest.gp import GP
import pandas as pd
import exoplanet as xo
import pymc3 as pm
import pymc3_ext as pmx
import ldtk
%pylab osx
,close all

s = """
planet  per      t0        T14    ror 
b       9.32414  2156.4239 2.305  2.60
c       15.50120 2155.4708 2.296  3.15
"""
trans = pd.read_csv(sio(s),sep='\s+',index_col=0)

Populating the interactive namespace from numpy and matplotlib


In [19]:
from ldtk import LDPSetCreator
sc = LDPSetCreator(teff=(4000,   400),    # Define your star, and the code
                   logg=(4.80, 0.20),    # downloads the uncached stellar
                   z=(-0.1, 0.1),
                   filters=[ldtk.kepler])

ps = sc.create_profiles()              
cq,eq = ps.coeffs_qd(do_mc=True)         # Estimate quadratic law coefficients
print(cq)
print(eq)

Need to download 189 files, approximately 63.13 MB


LDTk downloading uncached files:   0%|          | 0/189 [00:00<?, ?it/s]

[[0.43590115 0.26774386]]
[[0.01546328 0.01963826]]


In [42]:
u = array([0.43590115,0.26774386])
#uerr = array([0.01546328,0.01963826])

In [23]:
star = everest.Everest(206011691)
gp = GP(star.kernel, star.kernel_params, white=False)
gp.compute(star.apply_mask(star.time), star.apply_mask(star.fraw_err))
med = np.nanmedian(star.apply_mask(star.flux))
fgp, _ = gp.predict(star.apply_mask(star.flux) - med, star.time)
fwhite1 = (star.flux - fgp)
fwhite1 /= np.nanmedian(fwhite1)


clf()
figure(figsize=(20,4))
plot(star.apply_mask(star.time),star.apply_mask(star.flux),'.')
plot(star.time,fgp+med)

mask1 = star.mask.copy()

# Recompute GP with transits masked
for i, row in trans.iterrows():
    star.mask_planet(row.t0, row.per, dur=1.5 * row.T14/24.0)

star.compute()

gp = GP(star.kernel, star.kernel_params, white=False)
gp.compute(star.apply_mask(star.time), star.apply_mask(star.fraw_err))
med = np.nanmedian(star.apply_mask(star.flux))
fgp, _ = gp.predict(star.apply_mask(star.flux) - med, star.time)
fwhite2 = (star.flux - fgp)
fwhite2 /= np.nanmedian(fwhite2)

# Reset outlier and transit mask
plot(star.apply_mask(star.time),star.apply_mask(star.flux),'.')
plot(star.time,fgp+med)
star.transitmask = np.array([])
star.outmask = np.array([])
ylim(1.58e5, 1.60e5)

INFO  [everest.user.DownloadFile()]: Found cached file.
INFO  [everest.user.load_fits()]: Loading FITS file for 206011691.
INFO  [everest.basecamp.compute()]: Computing the model...


(158000.0, 160000.0)

## Compare whitened flux

In [26]:
plot(star.time,fwhite2,'.')
plot(star.apply_mask(star.time),star.apply_mask(fwhite2),'.')
ylim(0.999,1.001)

(0.999, 1.001)

## Perform a preliminary fit of whitened photometry 

In [43]:
t = star.apply_mask(star.time).astype('float64')
y = star.apply_mask(fwhite2).astype('float64')
y -= 1
yerr = 1.48 * median(np.abs(fwhite2-1))

print(yerr)
with pm.Model() as model:

    # The baseline flux
    mean = pm.Normal("mean", mu=0.0, sd=1.0)

    # The time of a reference transit for each planet
    t0 = pm.Normal("t0", mu=array(trans.t0), sd=1.0, shape=2)

    # The log period; also tracking the period itself
    logP = pm.Normal("logP", mu=np.log(array(trans.per)), sd=0.01, shape=2)
    period = pm.Deterministic("period", pm.math.exp(logP))

    # The Kipping (2013) parameterization for quadratic limb darkening paramters

    
    r = pm.Uniform(
        "r", lower=0.01, upper=0.1, shape=2, testval=np.array(trans.ror/100)
    )
    b = xo.distributions.ImpactParameter(
        "b", ror=r, shape=2, testval=np.random.rand(2)
    )

    # Set up a Keplerian orbit for the planets
    orbit = xo.orbits.KeplerianOrbit(period=period, t0=t0, b=b)

    # Compute the model light curve using starry
    light_curves = xo.LimbDarkLightCurve(u).get_light_curve(
        orbit=orbit, r=r, t=t,texp=0.020432
    )
    light_curve = pm.math.sum(light_curves, axis=-1) + mean

    # Here we track the value of the model light curve for plotting
    # purposes
    pm.Deterministic("light_curves", light_curves)

    # The likelihood function assuming known Gaussian uncertainty
    pm.Normal("obs", mu=light_curve, sd=yerr, observed=y)

    # Fit for the maximum a posteriori parameters given the simuated
    # dataset
    map_soln = pmx.optimize(start=model.test_point)

0.00010896290516807517
DEBUG [filelock.acquire()]: Attempting to acquire lock 140342813674704 on /Users/petigura/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG [filelock.acquire()]: Lock 140342813674704 acquired on /Users/petigura/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG [filelock.release()]: Attempting to release lock 140342813674704 on /Users/petigura/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG [filelock.release()]: Lock 140342813674704 released on /Users/petigura/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG [filelock.acquire()]: Attempting to acquire lock 140343028676208 on /Users/petigura/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG [filelock.acquire()]: Lock 140343028676208 acquired on /Users/petigura/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG [filelock.release()]: Attempting to release lock 1403430

DEBUG [filelock.release()]: Attempting to release lock 140343393693024 on /Users/petigura/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG [filelock.release()]: Lock 140343393693024 released on /Users/petigura/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG [filelock.acquire()]: Attempting to acquire lock 140343393806176 on /Users/petigura/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG [filelock.acquire()]: Lock 140343393806176 acquired on /Users/petigura/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG [filelock.release()]: Attempting to release lock 140343393806176 on /Users/petigura/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG [filelock.release()]: Lock 140343393806176 released on /Users/petigura/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG [filelock.acquire()]: Attempting to acquire lock 140343378606736 on /Users/peti

DEBUG [filelock.acquire()]: Attempting to acquire lock 140343393532320 on /Users/petigura/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG [filelock.acquire()]: Lock 140343393532320 acquired on /Users/petigura/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG [filelock.release()]: Attempting to release lock 140343393532320 on /Users/petigura/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG [filelock.release()]: Lock 140343393532320 released on /Users/petigura/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG [filelock.acquire()]: Attempting to acquire lock 140342811521376 on /Users/petigura/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG [filelock.acquire()]: Lock 140342811521376 acquired on /Users/petigura/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG [filelock.release()]: Attempting to release lock 140342811521376 on /Users/peti

optimizing logp for variables: [b, r, logP, t0, mean]


message: Desired error not necessarily achieved due to precision loss.
logp: 22180.814715920682 -> 22919.680672011604


In [44]:
,close all
fig,axL = subplots(nrows=2,sharex=True, sharey=True)
sca(axL[0])
plt.plot(t, y, ".k", ms=4, label="data")
for i, l in enumerate("bc"):
    plt.plot(
        t, map_soln["light_curves"][:, i], lw=1, label="planet {0}".format(l)
    )
plt.xlim(t.min(), t.max())
plt.ylabel("relative flux")
plt.xlabel("time [days]")
plt.legend(fontsize=10)
_ = plt.title("map model")

sca(axL[1])
plt.plot(t, y-map_soln["light_curves"].sum(-1), ".k", ms=4, label="data")


## Remove outliers to initial fit 

In [46]:
bout = abs(y-map_soln["light_curves"].sum(axis=-1)) > 5*yerr
y = y[~bout]
t = t[~bout]

with pm.Model() as model:

    # The baseline flux
    mean = pm.Normal("mean", mu=0.0, sd=1.0)

    # The time of a reference transit for each planet
    t0 = pm.Normal("t0", mu=array(trans.t0), sd=1.0, shape=2)

    # The log period; also tracking the period itself
    logP = pm.Normal("logP", mu=np.log(array(trans.per)), sd=0.01, shape=2)
    period = pm.Deterministic("period", pm.math.exp(logP))

    # The Kipping (2013) parameterization for quadratic limb darkening paramters
    r = pm.Uniform(
        "r", lower=0.01, upper=0.1, shape=2, testval=np.array(trans.ror/100)
    )
    b = xo.distributions.ImpactParameter(
        "b", ror=r, shape=2, testval=np.random.rand(2)
    )

    # Set up a Keplerian orbit for the planets
    orbit = xo.orbits.KeplerianOrbit(period=period, t0=t0, b=b)

    # Compute the model light curve using starry
    light_curves = xo.LimbDarkLightCurve(u).get_light_curve(
        orbit=orbit, r=r, t=t,texp=0.020432
    )
    light_curve = pm.math.sum(light_curves, axis=-1) + mean

    # Here we track the value of the model light curve for plotting
    # purposes
    pm.Deterministic("light_curves", light_curves)

    # The likelihood function assuming known Gaussian uncertainty
    pm.Normal("obs", mu=light_curve, sd=yerr, observed=y)

    # Fit for the maximum a posteriori parameters given the simuated
    # dataset
    map_soln = pmx.optimize(start=map_soln)

DEBUG [filelock.acquire()]: Attempting to acquire lock 140342820041440 on /Users/petigura/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG [filelock.acquire()]: Lock 140342820041440 acquired on /Users/petigura/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG [filelock.release()]: Attempting to release lock 140342820041440 on /Users/petigura/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.9.7-64/.lock
DEBUG [filelock.release()]: Lock 140342820041440 released on /Users/petigura/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.9.7-64/.lock


optimizing logp for variables: [b, r, logP, t0, mean]


message: Desired error not necessarily achieved due to precision loss.
logp: 24630.54723753681 -> 24630.547237536815


In [47]:
,close all
fig,axL = subplots(nrows=2,sharex=True, sharey=True)
sca(axL[0])
plt.plot(t, y, ".k", ms=4, label="data")
for i, l in enumerate("bc"):
    plt.plot(
        t, map_soln["light_curves"][:, i], lw=1, label="planet {0}".format(l)
    )
plt.xlim(t.min(), t.max())
plt.ylabel("relative flux")
plt.xlabel("time [days]")
plt.legend(fontsize=10)
_ = plt.title("map model")

sca(axL[1])
plt.plot(t, y-map_soln["light_curves"].sum(-1), ".k", ms=4, label="data")


In [53]:
df = pd.DataFrame(dict(t=t,f=y))
df['ferr'] = yerr
df.to_csv('k2-21_everest2-detrened-whitened.csv')

## Inspect phase folded light curves

In [51]:
%pylab osx
for n, letter in enumerate("bc"):
    plt.figure()

    # Get the posterior median orbital parameters
    p = map_soln["period"][n]
    t0 = map_soln["t0"][n]

    # Compute the median of posterior estimate of the contribution from
    # the other planet. Then we can remove this from the data to plot
    # just the planet we care about.
    other = map_soln["light_curves"][:, (n + 1) % 2]

    # Plot the folded data
    x_fold = (t - t0 + 0.5 * p) % p - 0.5 * p
    plt.errorbar(x_fold, y - other, yerr=yerr, fmt=".k", label="data", zorder=-1000)

    plt.legend(fontsize=10, loc=4)
    plt.xlim(-0.5 * p, 0.5 * p)
    plt.xlabel("time since transit [days]")
    plt.ylabel("relative flux")
    plt.title("planet {0}".format(letter))
    plt.xlim(-0.3, 0.3)

DEBUG [matplotlib.pyplot.switch_backend()]: Loaded backend MacOSX version unknown.
Populating the interactive namespace from numpy and matplotlib


In [55]:
np.random.seed(42)
with model:
    trace = pmx.sample(
        tune=1000,
        draws=2000,
        start=map_soln,
        cores=4,
        chains=4,
        target_accept=0.9,
        return_inferencedata=True,
    )

Multiprocess sampling (4 chains in 4 jobs)


INFO  [pymc3.sample()]: Multiprocess sampling (4 chains in 4 jobs)


NUTS: [b, r, logP, t0, mean]


INFO  [pymc3._print_step_hierarchy()]: NUTS: [b, r, logP, t0, mean]


WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 45 seconds.


INFO  [pymc3.sample()]: Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 45 seconds.


In [66]:
import arviz as az
az.summary(trace, var_names=["period", "t0", "r", "b"],round_to=5)['mean sd'.split()]

,mean,sd
period[0],9.32544,0.00041
period[1],15.50226,0.00072
t0[0],2156.42113,0.00146
t0[1],2155.47102,0.00145
r[0],0.02844,0.00048
r[1],0.03827,0.00063
b[0],0.77808,0.01130
b[1],0.87194,0.00544


In [62]:
az.summary?

In [ ]:
import corner
corner.corner(trace)


In [111]:
import pymc3 as pm
import theano.tensor as tt

np.random.seed(9485023)

with pm.Model() as model:

    # This part of the model is similar to the model in the `transit` tutorial
    mean = pm.Normal("mean", mu=0.0, sd=1.0)
    u_star = xo.QuadLimbDark("u_star")
    logr = pm.Uniform(
        "logr",
        lower=np.log(0.01),
        upper=np.log(0.1),
        shape=2,
        testval=np.log([0.04, 0.06]),
    )
    r = pm.Uniform(
        "r", lower=0.01, upper=0.1, shape=2, testval=np.array(trans.ror/100)
    )
    b = xo.distributions.ImpactParameter(
        "b", ror=r, shape=2, testval=np.random.rand(2)
    )

    # Now we have a parameter for each transit time for each planet:
    transit_times = []
    for i in range(2):
        transit_times.append(
            pm.Normal(
                "tts_{0}".format(i),
                mu=true_transit_times[i],
                sd=1.0,
                shape=len(true_transit_times[i]),
            )
        )

    # Set up an orbit for the planets
    orbit = xo.orbits.TTVOrbit(b=b, transit_times=transit_times)

    # It will be useful later to track some parameters of the orbit
    pm.Deterministic("t0", orbit.t0)
    pm.Deterministic("period", orbit.period)
    for i in range(2):
        pm.Deterministic("ttvs_{0}".format(i), orbit.ttvs[i])

    # The rest of this block follows the transit fitting tutorial
    light_curves = xo.LimbDarkLightCurve(u).get_light_curve(
        orbit=orbit, r=r, t=t, texp=texp
    )
    light_curve = pm.math.sum(light_curves, axis=-1) + mean
    pm.Deterministic("light_curves", light_curves)
    y = xo.eval_in_model(light_curve)
    y += yerr * np.random.randn(len(y))
    pm.Normal("obs", mu=light_curve, sd=yerr, observed=y)

    map_soln = model.test_point
    map_soln = xo.optimize(start=map_soln, vars=transit_times)
    map_soln = xo.optimize(start=map_soln, vars=[r, b])
    map_soln = xo.optimize(start=map_soln, vars=transit_times)
    map_soln = xo.optimize(start=map_soln)
    
    


    
    t0 = pm.Normal("t0", mu=array(trans.t0), sd=1.0, shape=2)

    # The log period; also tracking the period itself
    logP = pm.Normal("logP", mu=np.log(array(trans.per)), sd=0.01, shape=2)
    period = pm.Deterministic("period", pm.math.exp(logP))

    # The Kipping (2013) parameterization for quadratic limb darkening paramters

    # Set up a Keplerian orbit for the planets
    orbit = xo.orbits.KeplerianOrbit(period=period, t0=t0, b=b)

    # Compute the model light curve using starry
    light_curves = xo.LimbDarkLightCurve(u_star).get_light_curve(
        orbit=orbit, r=r, t=t,texp=0.020432
    )
    light_curve = pm.math.sum(light_curves, axis=-1) + mean

    # Here we track the value of the model light curve for plotting
    # purposes
    pm.Deterministic("light_curves", light_curves)

    # The likelihood function assuming known Gaussian uncertainty
    pm.Normal("obs", mu=light_curve, sd=yerr, observed=y)

    # Fit for the maximum a posteriori parameters given the simuated
    # dataset
    map_soln = pmx.optimize(start=map_soln)

In [126]:
dir(orbit)

['K0',
 'M0',
 'Omega',
 '__citations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_flip',
 '_get_acceleration',
 '_get_position',
 '_get_position_and_velocity',
 '_get_retarded_position',
 '_get_true_anomaly',
 '_get_velocity',
 '_rotate_vector',
 '_warp_times',
 'a',
 'a_planet',
 'a_star',
 'b',
 'cos_incl',
 'dcosidb',
 'ecc',
 'get_planet_acceleration',
 'get_planet_position',
 'get_planet_velocity',
 'get_radial_velocity',
 'get_relative_acceleration',
 'get_relative_angles',
 'get_relative_position',
 'get_relative_velocity',
 'get_star_acceleration',
 'get_star_position',
 'get_star_velocity',
 'in_transit',
 'incl',
 'jacobians',
 'm_planet',
 'm_star'